In [3]:
from main import FileExtractPhase

#spark = PysparkManager().CreateSparkSession()

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]") \
                .appName("PySparkShell") \
                .config("spark.jars", "postgresql-42.6.0.jar") \
                .getOrCreate()

23/08/03 12:02:32 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 172.29.37.254 instead (on interface eth0)
23/08/03 12:02:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/08/03 12:02:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [4]:
import configparser

config = configparser.ConfigParser(); config.read('config.ini')
database_url = config.get("DATABASE", "ConnectionUrl")

properties = {
    "driver": config.get("DATABASE", "driver"),
    "user": config.get("DATABASE", "ConnectionUser"),
    "password": config.get("DATABASE", "ConnectionPassword")
}

In [ ]:
table_name = "warehouse.test"

df = spark.read.jdbc(database_url, table_name, properties=properties)

In [5]:
df = FileExtractPhase(spark, directory="Tripdata-Source")
#df.cache()

In [11]:
%%time
df.printSchema()
df.count()

root
 |-- ride_id: string (nullable = true)
 |-- rideable_type: string (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- start_lat: double (nullable = true)
 |-- start_lng: double (nullable = true)
 |-- end_lat: double (nullable = true)
 |-- end_lng: double (nullable = true)
 |-- member_casual: string (nullable = true)

CPU times: user 5.84 ms, sys: 0 ns, total: 5.84 ms
Wall time: 98.1 ms


107288

In [8]:
df.cache()

23/08/03 12:12:22 WARN CacheManager: Asked to cache already cached data.


DataFrame[ride_id: string, rideable_type: string, started_at: timestamp, ended_at: timestamp, start_station_name: string, start_station_id: string, end_station_name: string, end_station_id: string, start_lat: double, start_lng: double, end_lat: double, end_lng: double, member_casual: string]

In [21]:
data_schema = df.schema

for field in data_schema.fields:
    null_count = df.filter(df[field.name].isNull()).count(); break
if null_count == 0 and len(df.columns) == 13 and len(df.columns) == 13:
    print("Data quality check passed")
else:
    print("Data quality check failed")

column_count = len(df.columns)
column_count

Data quality check passed


13

In [14]:
# SQL Select query
df.createOrReplaceTempView("mytable")
df.count()
spark.sql("select * from mytable").show(10)

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+-----------------+------------------+------------------+------------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|        start_lat|         start_lng|           end_lat|           end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+-----------------+------------------+------------------+------------------+-------------+
|0905B18B365C9D20| classic_bike|2023-01-28 09:18:10|2023-01-28 09:28:52|Hoboken Terminal ...|           HB101|       Hamilton Park|         JC009|40.73593758446329|-74.03030455112457|      40.727595966|     -74.044247311|       member|
|B4F0562B05CB5404|electric_bike|2023-01-23 20:10:12|2023

In [ ]:
from pyspark.sql.functions import col
#df = df.withColumn("started_at", col("started_at").cast("string "))
#df = df.withColumn("ended_at", col("ended_at").cast("string "))

df.limit(5).toPandas().head()


In [ ]:
spark.catalog.clearCache()

In [ ]:
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)
df.show()

In [2]:
spark.stop()